本章介绍表达式的构建，以及对不同数据类型的处理

- 布尔型
- 数字型
- 字符串型
- 日期和时间戳类型
- 空值处理
- 复杂类型
- 用户自定义函数

首先读取数据以便后续使用，inferSchema表示是否从数据中自动推导出schema

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python").getOrCreate()

In [2]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("./data/retail-data/by-day/2010-12-01.csv")

In [3]:
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



<h4>转换Spark类型</h4>

- lit函数能将其他语言的类型转换为对应的Spark类型数据

In [4]:
from pyspark.sql.functions import lit

In [5]:
df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

-- in SQL<br>
SELECT 5, "five", 5.0

<h4>处理布尔类型</h4>

- 布尔类型主要有四个要素组成：and, or, true, false

In [6]:
from pyspark.sql.functions import col

In [8]:
df.where(col("InvoiceNo") != 536365)\
.select("InvoiceNo", "Description")\
.show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [9]:
df.where("InvoiceNo = 536365").show(5, False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

In [10]:
df.where("InvoiceNo <> 536365").show(5, False)

+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                  |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|536366   |22633    |HAND WARMER UNION JACK       |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536366   |22632    |HAND WARMER RED POLKA DOT    |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536367   |84879    |ASSORTED COLOUR BIRD ORNAMENT|32      |2010-12-01 08:34:00|1.69     |13047.0   |United Kingdom|
|536367   |22745    |POPPY'S PLAYHOUSE BEDROOM    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
|536367   |22748    |POPPY'S PLAYHOUSE KITCHEN    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
+---------+---------+-----------------------------+--------+----

- 在Spark中，最好将多个Boolean表达式以链式连接的方式组合起来
- 因为即使你的表达式是顺序表达的，Spark还是会将过滤器合并为一条语句，并同时执行这些过滤器

In [11]:
from pyspark.sql.functions import instr

In [15]:
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



- 也可以不使用过滤器实现布尔表达式，而是利用其创造一个列

In [17]:
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
.where("isExpensive")\
.select("unitPrice", "isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



In [18]:
from pyspark.sql.functions import expr

In [20]:
df.withColumn("isExpensive", expr("Not UnitPrice <= 250"))\
.where("isExpensive")\
.select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



<h4>处理数值类型</h4>

- 在处理大数据时，过滤之后的常见任务就是计数
- 举个例子我们需要计算零售数据$(当前数量\times单位价格)^2+5$

In [19]:
from pyspark.sql.functions import pow

In [23]:
fabricatedQuantity = pow(col("Quantity")*col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(5)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
|   17850.0|          418.7156|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 5 rows



- 也可以用SQL表达式实现这些操作

In [25]:
df.selectExpr(
    "CustomerId",
    "(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity"
).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



-- in SQL<br>
SELECT customerId, (POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity FROM dfTable

- 取整，round函数会四舍五入，也可通过bround函数进行向下取整

In [28]:
from pyspark.sql.functions import lit, round, bround
df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)
df.select(round(lit("2.3")), bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows

+-------------+--------------+
|round(2.3, 0)|bround(2.5, 0)|
+-------------+--------------+
|          2.0|           2.0|
|          2.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [29]:
from pyspark.sql.functions import corr

In [33]:
df.select(corr("Quantity","UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



In [35]:
df.stat.corr("Quantity","UnitPrice")

-0.04112314436835551

-- in SQL<br>
SELECT corr("Quantity", "UnitPrice") FROM dfTable

- 另一个常见操作，是计算一列或一组列的汇总统计信息，可以用describe方法
- 它会计算所有数值类型的计数、平均值、标准差、最小值和最大值

In [36]:
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|        InvoiceDate|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|               3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|               null| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|               null|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             

In [37]:
from pyspark.sql.functions import count, mean, stddev_pop, min, max

In [41]:
quantileProbs = [0.5]
relError = 0.05
df.stat.approxQuantile("UnitPrice", quantileProbs, relError)

[2.51]

- 查看交叉项表和频繁项对

In [42]:
df.stat.crosstab("StockCode", "Quantity").show()

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

In [43]:
df.stat.freqItems(["StockCode", "Quantity"]).show()

+--------------------+--------------------+
| StockCode_freqItems|  Quantity_freqItems|
+--------------------+--------------------+
|[90214E, 20728, 2...|[200, 128, 23, 32...|
+--------------------+--------------------+



- 使用monotonically_increasing_id函数为每一行添加一个唯一的ID

In [46]:
from pyspark.sql.functions import monotonically_increasing_id
df.select("*",monotonically_increasing_id()).show(2)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|monotonically_increasing_id()|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|                            0|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|                            1|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------------------------+
only showing top 2 rows



<h4>处理字符串类型</h4>

- initcap函数每个单词首字母大写

In [48]:
from pyspark.sql.functions import initcap
df.select(initcap(col("Description"))).show(2, False)

+----------------------------------+
|initcap(Description)              |
+----------------------------------+
|White Hanging Heart T-light Holder|
|White Metal Lantern               |
+----------------------------------+
only showing top 2 rows



In [49]:
from pyspark.sql.functions import initcap
df.select(col("Description")).show(2, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
+----------------------------------+
only showing top 2 rows



- 将字符串转为大写或小写

In [55]:
from pyspark.sql.functions import lower, upper
df.select(upper(col("description")), lower(col("description"))).show(2)

+--------------------+--------------------+
|  upper(description)|  lower(description)|
+--------------------+--------------------+
|WHITE HANGING HEA...|white hanging hea...|
| WHITE METAL LANTERN| white metal lantern|
+--------------------+--------------------+
only showing top 2 rows



-- in SQL<br>
SELECT Description, lower(Description), Upper(Description) FROM dfTable

- 在字符串周围删除，添加空格
- ltrim, rtrim, trim分别对应删除左侧，右侧，两侧空格
- lpad, rpad分别向左侧和右侧添加空格，如果输入的数值小于字符串长度，则从字符串右侧删除字符

In [52]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim

In [74]:
df.select(
    ltrim(lit(" hello ")).alias("ltrim"),
    rtrim(lit(" hello ")).alias("rtrim"),
    trim(lit(" hello ")).alias("trim"),
    lpad(lit("hello"), 3, "x").alias("lp"),
    rpad(lit("hello"), 10, "x").alias("rp")).show(2)

+------+------+-----+---+----------+
| ltrim| rtrim| trim| lp|        rp|
+------+------+-----+---+----------+
|hello | hello|hello|hel|helloxxxxx|
|hello | hello|hello|hel|helloxxxxx|
+------+------+-----+---+----------+
only showing top 2 rows



<h4>正则表达式</h4>

- 帮助用户指定规则以从字符串中提取子串或替换子串
- Spark中两个关键函数：regexp_extract和regexp_replace
- 以下说明如何使用regexp_replace函数替换Description列中的颜色名

In [75]:
from pyspark.sql.functions import regexp_replace
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(
    regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"), col("Description")
).show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



-- SQL<br>
SELECT<br>
    regexp_replace(Description, 'BLACK|WHITE|RED|GREEN|BLUE', 'COLOR') AS<br>
    color_clean, Description<br>
FROM dfTable

- 还可以用其他字符替换给定字符，Spark提供translate操作来实现

In [78]:
from pyspark.sql.functions import translate
df.select(translate(col("Description"), "LEET", "1337"), col("Description")).show(2)

+----------------------------------+--------------------+
|translate(Description, LEET, 1337)|         Description|
+----------------------------------+--------------------+
|              WHI73 HANGING H3A...|WHITE HANGING HEA...|
|               WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
+----------------------------------+--------------------+
only showing top 2 rows



-- in SQL<br>
SELECT translate(Description, 'LEET', '1337'), Description FROM dfTable

- 也可以执行其他任务，比如取出第一个被提到的颜色
    - 0表示把整个正则表达式对应的结果全部返回
    - 1表示返回正则表达式中第一个() 对应的结果 以此类推 

In [87]:
from pyspark.sql.functions import regexp_extract
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(regexp_extract(col("Description"), extract_str, 1).alias("color_clean"), col("Description")).show(2)

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows



-- in SQL<br>
SELECT regexp_extract(Description, '(BLACK|WHITE|RED|GREEN|BLUE)', 1), Description FROM dfTable

- 我们有时并不需要提取字符，只想检查字符是否存在
- 可以在每列上使用contains方法

In [88]:
from pyspark.sql.functions import instr
containsBlack = instr(col("Description"), "BLACK") >= 1
containsWhite = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasSimpleColor", containsBlack | containsWhite)\
.where("hasSimpleColor")\
.select("DEscription").show(3, False)

+----------------------------------+
|DEscription                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



- 当有很多值需要检查时，我们利用Spark接收不定量参数的能力解决问题
- 

In [89]:
from pyspark.sql.functions import expr, locate

In [108]:
df.select(locate("RED", df.Description).cast("boolean")).show(10)

+--------------------------------------------+
|CAST(locate(RED, Description, 1) AS BOOLEAN)|
+--------------------------------------------+
|                                       false|
|                                       false|
|                                       false|
|                                       false|
|                                        true|
|                                       false|
|                                       false|
|                                       false|
|                                        true|
|                                       false|
+--------------------------------------------+
only showing top 10 rows



In [99]:
df.select("Description").show(10, False)

+-----------------------------------+
|Description                        |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
|SET 7 BABUSHKA NESTING BOXES       |
|GLASS STAR FROSTED T-LIGHT HOLDER  |
|HAND WARMER UNION JACK             |
|HAND WARMER RED POLKA DOT          |
|ASSORTED COLOUR BIRD ORNAMENT      |
+-----------------------------------+
only showing top 10 rows



In [114]:
simpleColors = ["black", "white", "red", "green", "blue"]
def color_locate(column, color_string):
    return locate(color_string.upper(), column).cast("boolean").alias("is_"+color_string)

selectedColumns = [color_locate(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("*"))

In [115]:
df.select(selectedColumns).show(2)

+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|is_black|is_white|is_red|is_green|is_blue|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   false|    true| false|   false|  false|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   false|    true| false|   false|  false|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



In [117]:
df.select(*selectedColumns).where(expr("is_red OR is_white")).select("Description").show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



<h4>处理日期和时间戳类型</h4>

- 当设置inferSchema为True时，Spark会自动推理日期和时间戳类型
- Spark使用的是Java日期和时间戳
- 以下首先获取当前日期和当前时间戳

In [122]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
.withColumn("today", current_date())\
.withColumn("now", current_timestamp())
dateDF.show()

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2021-07-10|2021-07-10 18:16:...|
|  1|2021-07-10|2021-07-10 18:16:...|
|  2|2021-07-10|2021-07-10 18:16:...|
|  3|2021-07-10|2021-07-10 18:16:...|
|  4|2021-07-10|2021-07-10 18:16:...|
|  5|2021-07-10|2021-07-10 18:16:...|
|  6|2021-07-10|2021-07-10 18:16:...|
|  7|2021-07-10|2021-07-10 18:16:...|
|  8|2021-07-10|2021-07-10 18:16:...|
|  9|2021-07-10|2021-07-10 18:16:...|
+---+----------+--------------------+



In [121]:
dateDF.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [123]:
dateDF.createOrReplaceTempView("dateTable")
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [124]:
from pyspark.sql.functions import date_add, date_sub
dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2021-07-05|        2021-07-15|
+------------------+------------------+
only showing top 1 row



-- in SQL<br>
SELECT date_sub(today, 5), date_add(today, 5) FROM dateTable

- 另一项常见任务是查看两个日期间的间隔时间
- datediff返回两个日期之间的天数，months_between给出两个日期间相隔的月数
- 该函数以指定的格式将字符串转为日期数据

In [126]:
from pyspark.sql.functions import datediff, months_between, to_date
dateDF.withColumn("week_ago", date_sub(col("today"), 7))\
.select(datediff(col("week_ago"), col("today"))).show(1)

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row



In [128]:
dateDF.select(
    to_date(lit("2016-01-01")).alias("start"),
    to_date(lit("2017-05-22")).alias("end")
).select(months_between(col("start"), col("end"))).show(1)

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



-- in SQL<br>
SELECT to_date('2016-01-01'), months_between('2016-01-01', '2017-01-01'), date_between('2016-01-01', '2017-01-01')<br>
FROM dateTable

- to_date以指定格式将字符串转为日期数据
- 如果Spark无法解析日期，它不会抛出错误，而只是返回null

In [129]:
dateDF.select(to_date(lit("2016-20-12")), to_date(lit("2017-12-11"))).show(1)

+-------------------+-------------------+
|to_date(2016-20-12)|to_date(2017-12-11)|
+-------------------+-------------------+
|               null|         2017-12-11|
+-------------------+-------------------+
only showing top 1 row



- 为了解决这个问题，我们使用两个函数：to_date和to_timestamp
- 前者可选用一种日期格式，后者强制要求使用一种日期格式

In [134]:
from pyspark.sql.functions import to_date
dateFormat = "yyyy-dd-MM"
cleanDateDF = spark.range(1).select(
    to_date(lit("2017-12-11"), dateFormat).alias("date"),
    to_date(lit("2017-20-12"), dateFormat).alias("date2")
)
cleanDateDF.createOrReplaceTempView("dateTable2")
cleanDateDF.show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



-- in SQL<br>
SELECT to_date(date, 'yyyy-dd-MM'), to_date(date2, 'yyyy-dd-MM') FROM dateTable2

- to_timestamp强制要求指定一种格式
- 写法一样

In [135]:
from pyspark.sql.functions import to_timestamp
cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

+------------------------------+
|to_timestamp(date, yyyy-dd-MM)|
+------------------------------+
|           2017-11-12 00:00:00|
+------------------------------+



 - 时间戳和日期之间的转换使用cast就行了

-- in SQL<br>
SELECT cast(to_date("2017-01-01", "yyyy-dd-MM") as timestamp)

- 日期/时间戳的比较只要确保使用同一种类型格式即可

In [136]:
cleanDateDF.filter(col("date2") > lit("2017-12-12")).show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



<h4>处理数据中的空值</h4>

- 建议始终使用null表示DataFrame中缺少或空的值，使用null值更有利于Spark优化
- coalesce返回参数列表中第一个非null值的字段，由于第一列无空值，所以合并第一第二列返回第一列

In [137]:
from pyspark.sql.functions import coalesce

In [146]:
df.select(coalesce(col("Description"), col("CustomerId"))).show(10,False)

+-----------------------------------+
|coalesce(Description, CustomerId)  |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
|SET 7 BABUSHKA NESTING BOXES       |
|GLASS STAR FROSTED T-LIGHT HOLDER  |
|HAND WARMER UNION JACK             |
|HAND WARMER RED POLKA DOT          |
|ASSORTED COLOUR BIRD ORNAMENT      |
+-----------------------------------+
only showing top 10 rows



<h4>ifnull, nullif, nvl, nvl2</h4>

- ifnull: 第一个值为空则选择第二个值
- nullif: 如果两个值相等则返回null
- nvl: 如果第一个值为null,返回第二个值
- nvl2: 如果第一个不为null,返回第二个值,否则返回最后一个指定值

-- in SQL<br>
SELECT<br>
ifnull(null, 'return_value'),<br>
nullif('value', 'value'),<br>
nvl(null, 'return_value'),<br>
nvl2('not_null', 'return_value', 'else_value')

<h4>drop</h4>

- drop用于删除包含null的行
- 若指定"any"作为参数,当存在一个值是null时,就删除该行
- 若指定"all"作为参数,只有当所有值为null时,才能删除该行

In [150]:
df.na.drop()
df.na.drop("any")
df.na.drop("all")
df.count()

3108

- 我们也可以指定几列,来对列进行删除空值的操作

In [151]:
df.na.drop("all", subset=["StockCode", "InvoiceNo"])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

<h4>fill</h4>

- 只针对空值进行的替换操作
- 想要指定多列,可以传入一个列名数组

In [154]:

df.na.fill("All Null values become this string")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [155]:
df.na.fill("all", subset=["StockCode", "InvoiceNo"])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

- 也可以使用映射,来实现,key是列名,value是想替换的值

In [156]:
fill_cols_vals = {"StockCode": 5, "Description": "No Value"}
df.na.fill(fill_cols_vals)

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

<h4>replace</h4>

- 除了使用drop和fill函数替换null值以外,还有不止针对空值的替换操作
- 可以根据当前值替换掉某列所有值
- 要求替换值与原始值类型相同

In [164]:
df.na.replace([""], ["UNKNOWN"], "Description")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

<h4>处理复杂类型</h4>

- Spark有三种复杂类型:结构体, 数组和Map

- 首先是结构体,结构体可以视为DataFrame中的DataFrame
- 当我们对多列使用结构体时,该结构体可以视为新的DataFrame
- 也可以通过用圆括号括起一组列来创建一个结构体

In [165]:
df.selectExpr("(Description, InvoiceNo) as complex", "*")
df.selectExpr("struct(Description, InvoiceNo) as complex", "*")

DataFrame[complex: struct<Description:string,InvoiceNo:string>, InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [166]:
from pyspark.sql.functions import struct
complexDF = df.select(struct("Description","InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")

In [167]:
complexDF.select("complex.Description")

DataFrame[Description: string]

In [168]:
complexDF.select(col("complex").getField("Description"))

DataFrame[complex.Description: string]

- 还可以用\*来查询结构体中的所有值,调出顶层DataFrame的所有列

In [169]:
complexDF.select("complex.*")

DataFrame[Description: string, InvoiceNo: string]

-- in SQL<br>
SELECT complex.\* FROM complexDF

<h4>数组</h4>

- 先看一个用例,读取Description列中的每行每个单词
- 通过使用split方法将Description列转为一个复杂类型

In [170]:
from pyspark.sql.functions import split
df.select(split(col("Description")," ")).show(2)

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
+-------------------------+
only showing top 2 rows



In [171]:
df.select(split(col("Description")," ").alias("array_col"))\
.selectExpr("array_col[0]").show(2)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows



-- in SQL<br>
SELECT split(Description, ' ') FROM dfTable<br>
SELECT split(Description, ' ')[0] FROM dfTable

<h4>数组长度</h4>

In [172]:
from pyspark.sql.functions import size

In [173]:
df.select(size(split(col("Description"), " "))).show(2)

+-------------------------------+
|size(split(Description,  , -1))|
+-------------------------------+
|                              5|
|                              3|
+-------------------------------+
only showing top 2 rows



<h4>array_contains</h4>

- 查询此数组是否包含某个人

In [175]:
from pyspark.sql.functions import array_contains
df.select(array_contains(split(col("Description"), " "), "WHITE")).show(2)

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|                                            true|
|                                            true|
+------------------------------------------------+
only showing top 2 rows



<h4>explode</h4>

- explode函数的输入参数为一个包含数组的列
- 该函数会为数组中的每个值创建一行
- 比如有一行为["Hello", "World"], "other col" -> 会变为两行,分别是"Hello", "other col"和"world", "other col"

In [176]:
from pyspark.sql.functions import split, explode

In [179]:
df.withColumn("splitted", split(col("Description"), " "))\
.withColumn("exploded", explode(col("splitted")))\
.select("Description", "InvoiceNo", "exploded").show(10, False)

+----------------------------------+---------+--------+
|Description                       |InvoiceNo|exploded|
+----------------------------------+---------+--------+
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |WHITE   |
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |HANGING |
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |HEART   |
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |T-LIGHT |
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |HOLDER  |
|WHITE METAL LANTERN               |536365   |WHITE   |
|WHITE METAL LANTERN               |536365   |METAL   |
|WHITE METAL LANTERN               |536365   |LANTERN |
|CREAM CUPID HEARTS COAT HANGER    |536365   |CREAM   |
|CREAM CUPID HEARTS COAT HANGER    |536365   |CUPID   |
+----------------------------------+---------+--------+
only showing top 10 rows



-- in SQL<br>
SELECT Description, InvoiceNo, exploded<br>
FROM (SELECT \*, split(Description, " ") as splitted FROM dfTable)<br>
LATERAL VIEW explode(splitted) as exploded

<h4>map</h4>

- map映射通过map函数构建两列内容的键值对映射形式

In [185]:
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")).show(5, False)

+-----------------------------------------------+
|complex_map                                    |
+-----------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER -> 536365} |
|{WHITE METAL LANTERN -> 536365}                |
|{CREAM CUPID HEARTS COAT HANGER -> 536365}     |
|{KNITTED UNION FLAG HOT WATER BOTTLE -> 536365}|
|{RED WOOLLY HOTTIE WHITE HEART. -> 536365}     |
+-----------------------------------------------+
only showing top 5 rows



-- in SQL<br>
SELECT map(Description, InvoiceNo) as complex_map FROM dfTable<br>
WHERE Description IS NOT NULL

In [186]:
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
.selectExpr("complex_map['WHITE METAL LANTERN']").show(5)

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                            null|
|                          536365|
|                            null|
|                            null|
|                            null|
+--------------------------------+
only showing top 5 rows



- 对map类型使用explode会让key和value分离

In [190]:
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
.selectExpr("explode(complex_map)").show(5, False)

+-----------------------------------+------+
|key                                |value |
+-----------------------------------+------+
|WHITE HANGING HEART T-LIGHT HOLDER |536365|
|WHITE METAL LANTERN                |536365|
|CREAM CUPID HEARTS COAT HANGER     |536365|
|KNITTED UNION FLAG HOT WATER BOTTLE|536365|
|RED WOOLLY HOTTIE WHITE HEART.     |536365|
+-----------------------------------+------+
only showing top 5 rows



<h4>处理JSON类型</h4>



In [194]:
jsonDF = spark.range(1).selectExpr("""
'{"myJSONKey": {"myJSONValue": [1,2,3]}}' as jsonString
""")

In [199]:
jsonDF.show(truncate=False)

+---------------------------------------+
|jsonString                             |
+---------------------------------------+
|{"myJSONKey": {"myJSONValue": [1,2,3]}}|
+---------------------------------------+



In [200]:
from pyspark.sql.functions import get_json_object, json_tuple

In [203]:
jsonDF.select(
    get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]") as "column",
    json_tuple(col("jsonString"), "myJSONKey")
).show(2)

SyntaxError: invalid syntax (<ipython-input-203-5c2f320081ea>, line 2)

In [204]:
jsonDF.selectExpr(
    "json_tuple(jsonString, '$.myJSONKey.myJSONValue[1]') as column"
).show(2)

+------+
|column|
+------+
|  null|
+------+



<h4>用户自定义函数</h4>

- 用户自定义函数(UDF)让用户可以使用Python编写自己的自定义转换操作
- 这些函数被注册为SparkSession或者Context的临时函数
- 首先编写一个power3函数,该函数接收一个数字并返回它的三次幂

In [208]:
udfExampleDF = spark.range(5).toDF("num")
# udfExampleDF.show()

In [207]:
def power3(double_value):
    return double_value ** 3
power3(2.0)

8.0

- 函数创建以后需要在Spark上注册UDF,以便在所有的工作机器上使用它们
- Spark将在驱动进程上序列化该函数, 并通过网络传输给所有的进程
- 如果该函数时Scala或Java写的,则可以在Java虚拟机中使用,这可能会导致性能的下降
- 如果该函数时Python编写的,Spark会在worker上启动一个Python进程,将所有数据序列化为Python可解释的格式
- 在Python进程中对该数据逐行执行函数

1. Python用户在驱动器中自定义函数
2. 函数序列化并发送给工作节点
3. Spark启动Python进程并发送数据
4. Python进程返回结果

 <h4>注册UDF</h4>

In [209]:
from pyspark.sql.functions import udf
power3udf = udf(power3)

In [210]:
from pyspark.sql.functions import col
udfExampleDF.select(power3udf(col("num"))).show(5)

Py4JJavaError: An error occurred while calling o1373.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 164.0 failed 1 times, most recent failure: Lost task 0.0 in stage 164.0 (TID 493) (26.26.26.1 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:863)
	at org.apache.spark.rdd.RDD.$anonfun$mapPa

spark.udf.register("power3", power3\(_:Double):Double)<br>
udfExampleDF.selectExpr("power3(num)").show(2)